In [180]:
%load_ext autoreload
%autoreload 2

import numpy as np
from investment_planner import calculateTransitionPropabilitiesForAllPorfolios, generateGrid, calculateTransitionPropabilities, get_portfolios_strategies
from scipy.stats import norm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 3
h = 1

values = [0,0,100,100]

#propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, cost, h)
propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]])
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)
#values.sum(2)


[[[0.451 0.334 0.197 0.018]
  [0.271 0.27  0.258 0.2  ]
  [0.255 0.256 0.253 0.237]]

 [[0.14  0.307 0.366 0.187]
  [0.221 0.254 0.266 0.259]
  [0.24  0.251 0.255 0.254]]

 [[0.017 0.105 0.244 0.634]
  [0.177 0.234 0.268 0.322]
  [0.227 0.246 0.256 0.271]]

 [[0.001 0.017 0.075 0.908]
  [0.14  0.211 0.262 0.387]
  [0.215 0.241 0.257 0.288]]]


array([49. , 55.3, 87.8, 98.3])

In [127]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057], [0.0886, 0.1954]])
W0 = 100
T = 12
#infusions = [0,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000]
infusions = np.zeros(13)
goals = [0,0,0,0,0,0,0,100,0,0,0,0,150]
iMax = 281

grid = generateGrid(W0,T,iMax, infusions, goals, portfolios[0,0], portfolios[0,1], portfolios[2,0], portfolios[2,1])
grid = grid.T

In [114]:
W0 = 100
meanMax = 0.0886
stdMin = 0.0374
stdMax = 0.1954
t =11

W0*np.exp((meanMax-(stdMin**2/2))*t + 3*stdMax*np.sqrt(t))

1837.722768128645

In [ ]:
k = np.array([[105,100],[110,200]])
wealthInT = np.array([90,95,100,105,110])
Wk = np.tile(wealthInT, (2,1))
ki = np.tile(k, (2, 5,1))
#np.where(Wk >= k[:,0], k[:,1], 0)
ki

array([[[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]],

       [[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]]])

In [ ]:
WT1 = np.array([90,95,100,105,110])
WT1P = WT1 * [0.2,0.35, 0.2 ,0.15,0.1]
WT1P.reshape((5,1))

array([[18.  ],
       [33.25],
       [20.  ],
       [15.75],
       [11.  ]])

In [ ]:
arr = np.array([[1,2,3,4],[1,2,3]])
arr2 = np.array([1,2])
result = arr*arr2
result

C:\Users\matsz\AppData\Local\Temp\ipykernel_26768\4293703242.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([[1,2,3,4],[1,2,3]])


array([list([1, 2, 3, 4]), list([1, 2, 3, 1, 2, 3])], dtype=object)

In [ ]:
def get_ef_port_analytically(rets, covm, targ):
    """Solve for the efficient frontier weights for a given expected return
    vector `rets`, covariance matrix `covm`, and expected portfolio return
    `targ`.
    """
    N = rets.shape[0]
    u = np.array([targ, 1])[:, None]
    U = np.vstack([rets, np.ones_like(rets)]).T

    covm_inv = np.linalg.solve(covm, np.eye(N))
    M        = U.T @ covm_inv @ U
    M_inv    = np.linalg.solve(M, np.eye(2))
    weights  = covm_inv @ U @ M_inv @ u

    return weights


In [ ]:
cov_tab = np.array([[0.0002, -0.00],
                    [-0.00, 0.0013]])
means = np.array([0.0307, 0.1094])

mu = 0.0744
std = 0.0712

w = get_ef_port_analytically(means,cov_tab,mu)

w

array([[0.44472681],
       [0.55527319]])

In [79]:
arr = np.empty((3,3))

arr[:] = np.nan
arr




array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [125]:
mean = 0.0886
std = 0.1954
W0 = 105
W1 = np.array([105,110,115])
h= 1
np.round(norm.pdf((np.log(W1/(W0))-(mean-0.5*std**2)*h)/(std*np.sqrt(h))),3)

#100 [0.39670583, 0.39547966, 0.37397359]
#105 [0.37448256, 0.39619069, 0.39654302]


array([0.374, 0.396, 0.397])

In [136]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 50
h = 1

values = [0,0,100,100]

propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, cost, h)
''' propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]]) '''
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)

[[[0.    0.    0.    1.   ]
  [0.027 0.082 0.162 0.73 ]
  [0.152 0.209 0.253 0.386]]

 [[0.    0.    0.    1.   ]
  [0.021 0.069 0.146 0.765]
  [0.145 0.204 0.251 0.4  ]]

 [[0.    0.    0.    1.   ]
  [0.016 0.058 0.131 0.795]
  [0.138 0.199 0.249 0.414]]

 [[0.    0.    0.    1.   ]
  [0.012 0.049 0.117 0.821]
  [0.131 0.194 0.247 0.427]]]


array([100., 100., 100., 100.])

In [231]:
%precision 3

def get_goals_strategies(goals, Wt, VTk0):
    strategies = np.zeros((len(goals), len(Wt)))
    for k in range(len(goals)):
        for i in range(len(Wt)):
            strategies[k,i] = k+1 if Wt[i] > goals[k][0] else 0
    return strategies

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,100,105])
WT1 = [100,103,105,110]
VT1 = [0,0,100,100]
c = 50
#goals = [[50, 80], [30, 50],]
goals = goals = [[49,40],[50, 80],[60, 50]]
VTc0 = np.array([29.014, 29.55, 77.286, 95.997])

prob = calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,3,0)

strategies, VT = get_portfolios_strategies(VT1,prob)

#print(prob*VT1)
print(strategies)
print(VT) 

# VT while cost 0 [29.014 29.55  77.286 95.997]

goal_strategies = get_goals_strategies(goals,WT,VTc0)

goal_strategies


[2 2 0 0]
[29.014 29.55  77.286 95.997]


array([[0., 1., 1., 1.],
       [0., 0., 2., 2.],
       [0., 0., 3., 3.]])

## TODO
- sprawdzić wrażliwość na rożne inputy
- W0 jest większe niż cele